# instructions for setting up environment:
# Create a new environment:

conda create -n rsoxs-databroker python=3.7
conda activate rsoxs-databroker

# Clone the tiled and databroker repos:

git clone https://github.com/bluesky/tiled
git clone https://github.com/bluesky/databroker

# Pip install tiled...

pip install -e tiled[complete]

# ... and the tiled-refactor branch of databroker

cd databroker/
git fetch
git checkout tiled-refactor
pip install -e .[complete]
cd ..

# Pip install some other packages:

pip install ipython area-detector-handlers

# conda install more stuff

conda install -c pyviz hvplot jupyterlab

# setup jupyterlab on ws24

## create a config file for jupyterlab

# in your home directory somewhere (usually ~/.jupyter/

jupyter-lab --generate-config

# this creates a jupyter_lab_config.py file in this directory  edit this file so:

c.ServerApp.allow_origin = '*'
c.ServerApp.allow_remote_access = True

# create a password for your jupyter-lab

jupyter-lab password

# now start up your jupyter server




In [ ]:
# Get the rsoxs catalog
import os
os.environ["TILED_SITE_PROFILES"] = '/nsls2/software/etc/tiled/profiles'
from matplotlib.colors import LogNorm
from tiled.client import from_profile
from databroker.queries import RawMongo
import xarray as xr
import copy
import numpy as np
from matplotlib import pyplot as plt
#c = from_profile('rsoxs')
import holoviews as hv
import hvplot.xarray
import datetime

import pandas as pd

to set up pyhyper:

install your ws24 ssh key into your github account

cd ~/
git clone git@github.com:usnistgov/PyHyperScattering.git

you should place this notebook and the SST1_WAXS_mask.hdf5 and SST1-SAXS_mask.hdf5 files in your home dir (or wherever else you can find them).

You will also need to 

conda install pyfai pyopencl

(maybe others too)

have fun!


In [ ]:
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('./PyHyperScattering/PyHyperScattering/')
from SST1RSoXSDB import SST1RSoXSDB
from PFEnergySeriesIntegrator import PFEnergySeriesIntegrator
rsoxsload = SST1RSoXSDB(corr_mode='none')

In [ ]:
# Do a custom search

#c = from_profile('rsoxs')
#q = RawMongo(start={"plan_name": "short_carbon_scan_nd"})
#results = c.search(q)

results = rsoxsload.runSearch(start={'plan_name':'short_carbon_scan_nd','institution':'NIST'})

In [ ]:
results

In [ ]:
run = results[-1100]

In [ ]:
itp = rsoxsload.loadRun(results[-10],dims=['energy'])

In [ ]:
integ = PFEnergySeriesIntegrator(maskmethod='nika',maskpath='SST1-SAXS_mask.hdf',geomethod='template_xr',template_xr=itp)

In [ ]:
integratedimages = integ.integrateImageStack(itp)

In [ ]:
integratedimages.unstack('system').sel(energy=270).hvplot(logz=True,clim=(20, 5000),cmap='terrain')

In [ ]:
integratedimages.unstack('system').hvplot(groupby='energy',dynamic=False,logz=True,clim=(20, 5000),cmap='terrain')

In [ ]:
import RSoXS

RSoXS.AR(integratedimages.unstack('system')).hvplot(clim=(-1,1),cmap='bwr')

In [ ]:
# make a color map for plotting
cmap = copy.copy(plt.cm.terrain)
cmap.set_under('black') # color below the minimum value
cmap.set_over('purple') # color above
cmap.set_bad('black') # color for negative numbers (nans on log scale)

In [ ]:
# use xarray's default facetgrip plot to look at everything at once
itp.unstack('system').coarsen(pix_x=8,pix_y=8,boundary='trim',coord_func='mean').mean().plot(col='energy',col_wrap=8,norm=LogNorm(vmin=10, vmax=6000),cmap=cmap)

In [ ]:
itp.sel(energy=285,method='nearest')

In [ ]:
%%time
# look at one energy using holoviews
itp.sel(energy=285,method='nearest').hvplot.image(data_aspect=1,dynamic=False,logz=True,clim=(20, 5000),cmap='terrain')

In [ ]:
%%time
# look at all of the images with a nice automatic slider
itp.unstack('system').coarsen(pix_x=4,pix_y=4,boundary='trim',coord_func='mean').mean().hvplot(groupby='energy',data_aspect=1,dynamic=False,logz=True,clim=(20, 5000),cmap='terrain')
# to add: http://holoviews.org/reference/streams/bokeh/linked_pointer_crosssection.html

In [ ]:
%%time
itp.unstack('system').coarsen(pix_x=8,pix_y=8,boundary='trim',coord_func='mean').mean().hvplot(by='energy',dynamic=False,logz=True,clim=(20, 50000),cmap='terrain',height=150,width=200).grid('energy')